In [1]:
import umap
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
res_folders=os.listdir('../../results/')
model_folder='/home/mara/multitask_adversarial/results/NCOUNT_822/'
CONCEPT=['ncount']
import keras
keras.__version__
from sklearn.metrics import accuracy_score
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(0)# str(hvd.local_rank())
keras.backend.set_session(tf.Session(config=config))
verbose=1 
init=tf.global_variables_initializer() #initialize_all_variables()
sess=tf.Session()
sess.run(init)
#reducer = umap.UMAP()
#reducer = reducer.fit(feats)
#embedding = reducer.transform(feats)

Using TensorFlow backend.


In [2]:
model_folder='/home/mara/multitask_adversarial/results/NCOUNT_1139/'

In [3]:
## Loading OS libraries to configure server preferences
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import setproctitle
SERVER_NAME = 'ultrafast'
EXPERIMENT_TYPE='test_domain'
import time
import sys
import shutil
## Adding PROCESS_UC1 utilities
sys.path.append('../../lib/TASK_2_UC1/')
from models import *
from util import otsu_thresholding
from extract_xml import *
from functions import *                   
sys.path.append('../../lib/')
from mlta import *
import math
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(0)# str(hvd.local_rank())
keras.backend.set_session(tf.Session(config=config))

verbose=1 
"""loading dataset files"""
#rank = MPI.COMM_WORLD.rank
cam16 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/cam16_500/patches.h5py',  'r', libver='latest', swmr=True)
all500 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/all500/patches.h5py',  'r', libver='latest', swmr=True)
extra17 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/extra17/patches.h5py','r', libver='latest', swmr=True)
tumor_extra17=hd.File('/home/mara/adversarialMICCAI/data/ultrafast/1129-1155/patches.h5py', 'r', libver='latest', swmr=True)
test2 = hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/ultrafast/test_data2/patches.hdf5', 'r', libver='latest', swmr=True)
pannuke= hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/pannuke/patches_fix.hdf5', 'r', libver='latest', swmr=True)

global datasetss
datasetss={'cam16':cam16,'all500':all500,'extra17':extra17, 'tumor_extra17':tumor_extra17, 'test_data2': test2, 'pannuke':pannuke}

global concept_db
concept_db = hd.File('../../data/normalized_cmeasures/concept_values_def.h5py','r')

#SYSTEM CONFIGS 
CONFIG_FILE = '../../doc/config.cfg'
COLOR = True
BATCH_SIZE = 32

# SAVE FOLD
f=open(model_folder+"/seed.txt","r")
seed=1001#int(f.read())
if verbose:  print(seed)
#f.write(str(seed))
f.close()


# SET PROCESS TITLE
setproctitle.setproctitle('{}'.format(EXPERIMENT_TYPE))

# SET SEED
np.random.seed(seed)
tf.set_random_seed(seed)

# DATA SPLIT CSVs 
train_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/train_shuffle.csv', 'r') # How is the encoding of .csv files ?
val_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/val_shuffle.csv', 'r')
test_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/test_shuffle.csv', 'r')
train_list=train_csv.readlines()
val_list=val_csv.readlines()
test_list=test_csv.readlines()
test2_csv = open('/mnt/nas2/results/IntermediateResults/Camelyon/test2_shuffle.csv', 'r')
test2_list=test2_csv.readlines()
test2_csv.close()
train_csv.close()
val_csv.close()
test_csv.close()
data_csv=open('../../doc/pannuke_data_shuffle.csv', 'r')
data_list=data_csv.readlines()
data_csv.close()

# STAIN NORMALIZATION
def get_normalizer(patch, save_folder='../../results/'):
    normalizer = ReinhardNormalizer()
    normalizer.fit(patch)
    np.save('{}/normalizer'.format(save_folder),normalizer)
    np.save('{}/normalizing_patch'.format(save_folder), patch)
    #print('Normalisers saved to disk.')
    return normalizer
def normalize_patch(patch, normalizer):
    return np.float64(normalizer.transform(np.uint8(patch)))

global normalizer
db_name, entry_path, patch_no = get_keys(data_list[0])
normalization_reference_patch = datasetss[db_name][entry_path][patch_no]
normalizer = get_normalizer(normalization_reference_patch, save_folder='../../results/')

"""
Batch generators: 
They load a patch list: a list of file names and paths. 
They use the list to create a batch of 32 samples. 
"""

# Retrieve Concept Measures
def get_concept_measure(db_name, entry_path, patch_no, measure_type=''):
    if measure_type=='domain':
        return get_domain(db_name, entry_path)
    path=db_name+'/'+entry_path+'/'+str(patch_no)+'/'+measure_type.strip(' ')
    try:
        cm=concept_db[path][0]
        return cm
    except:
        print("[ERR]: {}, {}, {}, {} with path {}".format(db_name, entry_path, patch_no, measure_type, path))
        #import pdb; pdb.set_trace()
        return 1.
    
# BATCH GENERATORS
import keras.utils
class DataGenerator(keras.utils.Sequence):
    def __init__(self, patch_list, concept=CONCEPT, batch_size=32, shuffle=True, data_type=0):
        self.batch_size=batch_size
        self.patch_list=patch_list
        self.shuffle=shuffle
        self.concept = concept
        self.data_type=data_type
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.patch_list)/self.batch_size))
    
    def __getitem__(self, index):
        indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        patch_list_temp=[self.patch_list[k] for k in indexes]
        self.patch_list_temp=patch_list_temp
        return self.__data_generation(self), None
    
    def get(self, index):
        indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        patch_list_temp=[self.patch_list[k] for k in indexes]
        self.patch_list_temp=patch_list_temp
        return self.__data_generation(self), None
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.patch_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, patch_list_temp):
        patch_list_temp=self.patch_list_temp
        batch_x=np.zeros((len(patch_list_temp), 224,224,3))
        batch_y=np.zeros(len(patch_list_temp))
        i=0
        for line in patch_list_temp:
            db_name, entry_path, patch_no = get_keys(line)
            patch=datasetss[db_name][entry_path][patch_no]
            patch=normalize_patch(patch, normalizer)
            patch=keras.applications.inception_v3.preprocess_input(patch) 
            label = get_class(line, entry_path) 
            if self.data_type!=0:
                label=get_test_label(entry_path)
            batch_x[i]=patch
            batch_y[i]=label
            i+=1
        generator_output=[batch_x, batch_y]
        for c in self.concept:
            batch_concept_values=np.zeros(len(patch_list_temp))
            i=0
            for line in patch_list_temp:
                db_name, entry_path, patch_no = get_keys(line)
                batch_concept_values[i]=get_concept_measure(db_name, entry_path, patch_no, measure_type=c)
                i+=1
            if c=='domain':
                    batch_concept_values=keras.utils.to_categorical(batch_concept_values, num_classes=7)
            generator_output.append(batch_concept_values)
        return generator_output
    

1001
Using brightness standardization


In [4]:
#import matplotlib as mpl   
#mpl.use('Agg')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger('tensorflow').disabled = True
from keras import *
import setproctitle
SERVER_NAME = 'ultrafast'
import time
import sys
import shutil
## Adding PROCESS_UC1 utilities
sys.path.append('../../lib/TASK_2_UC1/')
from models import *
from util import otsu_thresholding
from extract_xml import *
from functions import *                   
sys.path.append('../../lib/')
from mlta import *
import math
import keras.callbacks as callbacks
from keras.callbacks import Callback

In [5]:
keras.backend.clear_session()
"""         
Building guidable model 
"""
def get_baseline_model(hp_lambda=0., domain=False, c_list=[]):
    base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
    layers_list=['conv2d_92', 'conv2d_93', 'conv2d_88', 'conv2d_89', 'conv2d_86']
    #layers_list=[]
    for i in range(len(base_model.layers[:])):
        layer=base_model.layers[i]
        if layer.name in layers_list:
            print layer.name
            layer.trainable=True
        else:
            layer.trainable = False
    feature_output=base_model.layers[-1].output
    gap_layer_output = keras.layers.GlobalAveragePooling2D()(feature_output)
    feature_output = Dense(2048, activation='relu', name='finetuned_features1',kernel_regularizer=keras.regularizers.l2(0.01))(gap_layer_output) 
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    feature_output = Dense(512, activation='relu', name='finetuned_features2',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    feature_output = Dense(256, activation='relu', name='finetuned_features3',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    grl_layer=GradientReversal(hp_lambda=hp_lambda)
    feature_output_grl = grl_layer(feature_output)
    if domain:
        domain_adversarial = keras.layers.Dense(7, activation = keras.layers.Activation('softmax'), name='domain_adversarial')(feature_output_grl)
    finetuning = Dense(1,name='predictions')(feature_output)
    ## here you need to check how many other concepts you have apart from domain adversarial
    # then you add one layer per each. 
    if domain:
        output_nodes=[finetuning, domain_adversarial]
    else: 
        output_nodes=[finetuning]
    for c in c_list:
        if c!='domain':
            concept_layer=  keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='extra_{}'.format(c.strip(' ')))(feature_output)
            output_nodes.append(concept_layer)
    model = Model(input=base_model.input, output=output_nodes)
    model.grl_layer=grl_layer
    return model
""" 
Get trainable model with Hepistemic Uncertainty Weighted Loss 
"""
# Custom loss layer
from keras.initializers import Constant
class CustomMultiLossLayer(Layer):
    def __init__(self, nb_outputs=2, **kwargs):
        self.nb_outputs = nb_outputs
        self.is_placeholder = True
        super(CustomMultiLossLayer, self).__init__(**kwargs)
        
    def build(self, input_shape=None):
        self.log_vars = []
        for i in range(self.nb_outputs):
            self.log_vars += [self.add_weight(name='log_var' + str(i), shape=(1,), initializer=Constant(0.), trainable=True)]
        super(CustomMultiLossLayer, self).build(input_shape)
    
    def multi_loss(self,  ys_true, ys_pred):
        #print len(ys_true)
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        i=0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision =K.exp(-log_var[0]) ###MODIFICATION HERE
            if i==0:
                pred_loss = bbce(y_true, y_pred)
                term = precision*pred_loss + 0.5 * log_var[0]  
                #term=tf.Print(term, [term], 'bbce: ')
            else:
                pred_loss = keras_mse(y_true, y_pred)
                #pred_loss=tf.Print(pred_loss, [pred_loss], 'MSE: ')
                term = 0.5 * precision * pred_loss + 0.5 * log_var[0]
                #term=tf.Print(term, [term], 'MSE: ')
            loss+=term
            term = 0.
            i+=1
        return K.mean(loss)
    
    def call(self, inputs):
        ys_true = inputs[:self.nb_outputs]
        ys_pred = inputs[self.nb_outputs:]
        loss = self.multi_loss(ys_true, ys_pred)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return K.concatenate(inputs, -1)
def get_trainable_model(baseline_model):
    inp = keras.layers.Input(shape=(224,224,3,), name='inp')
    y1_pred, y2_pred = baseline_model(inp)
    y1_true=keras.layers.Input(shape=(1,),name='y1_true')
    y2_true=keras.layers.Input(shape=(1,),name='y2_true')
    out = CustomMultiLossLayer(nb_outputs=2)([y1_true, y2_true, y1_pred, y2_pred])
    return Model(input=[inp, y1_true, y2_true], output=out)

""" Get trainable model with Hepistemic Uncertainty Weighted Loss """


""" 
LOSS FUNCTIONS
"""
def keras_mse(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.mean_squared_error(y_true, y_pred))

def bbce(y_true, y_pred):
    # we use zero weights to set the loss to zero for unlabeled data
    verbose=0
    zero= tf.constant(-1, dtype=tf.float32)
    where = tf.not_equal(y_true, zero)
    where = tf.reshape(where, [-1])
    indices=tf.where(where) #indices where the item of y_true is NOT -1
    indices = tf.reshape(indices, [-1])
    sliced_y_true = tf.nn.embedding_lookup(y_true, indices)
    sliced_y_pred = tf.nn.embedding_lookup(y_pred, indices)
    n1 = tf.shape(indices)[0] #number of train images in batch
    batch_size = tf.shape(y_true)[0]
    n2 = batch_size - n1 #number of test images in batch
    sliced_y_true = tf.reshape(sliced_y_true, [n1, -1])
    n1_ = tf.cast(n1, tf.float32)
    n2_ = tf.cast(n2, tf.float32)
    multiplier = (n1_+ n2_) / n1_
    zero_class = tf.constant(0, dtype=tf.float32)
    where_class_is_zero=tf.cast(tf.reduce_sum(tf.cast(tf.equal(sliced_y_true, zero_class), dtype=tf.float32)), dtype=tf.float32)
    if verbose:
        where_class_is_zero=tf.Print(where_class_is_zero,[where_class_is_zero],'where_class_is_zero: ')
    class_weight_zero = tf.cast(tf.divide(n1_, 2. * tf.cast(where_class_is_zero, dtype=tf.float32)+0.001), dtype=tf.float32)
    
    if verbose:
        class_weight_zero=tf.Print(class_weight_zero,[class_weight_zero],'class_weight_zero: ')
    one_class = tf.constant(1, dtype=tf.float32)
    where_class_is_one=tf.cast(tf.reduce_sum(tf.cast(tf.equal(sliced_y_true, one_class), dtype=tf.float32)), dtype=tf.float32)
    if verbose:
        where_class_is_one=tf.Print(where_class_is_one,[where_class_is_one],'where_class_is_one: ')
        n1_=tf.Print(n1_,[n1_],'n1_: ')
    class_weight_one = tf.cast(tf.divide(n1_, 2. * tf.cast(where_class_is_one,dtype=tf.float32)+0.001), dtype=tf.float32)
    class_weight_zero =  tf.constant(23477.0/(23477.0+123820.0), dtype=tf.float32)
    class_weight_one =  tf.constant(123820.0/(23477.0+123820.0), dtype=tf.float32)
    A = tf.ones(tf.shape(sliced_y_true), dtype=tf.float32) - sliced_y_true 
    A = tf.scalar_mul(class_weight_zero, A)
    B = tf.scalar_mul(class_weight_one, sliced_y_true) 
    class_weight_vector=A+B
    ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=sliced_y_true,logits=sliced_y_pred)
    ce = tf.multiply(class_weight_vector,ce)
    return tf.reduce_mean(ce)

from keras.initializers import Constant
global domain_weight
global main_task_weight

class CustomMultiLossLayer(Layer):
    def __init__(self, new_folder='', nb_outputs=2, **kwargs):
        self.nb_outputs = nb_outputs
        self.is_placeholder = True
        super(CustomMultiLossLayer, self).__init__(**kwargs)
        
    def build(self, input_shape=None):
        # initialise log_vars
        self.log_vars = []
        for i in range(self.nb_outputs):
            self.log_vars += [self.add_weight(name='log_var' + str(i), shape=(1,),
                                              initializer=Constant(0.), trainable=True)]
        super(CustomMultiLossLayer, self).build(input_shape)
    """
    def multi_loss(self, ys_true, ys_pred):
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision = K.exp(-log_var[0])
            loss += K.sum(precision * (y_true - y_pred)**2. + log_var[0], -1)
        return K.mean(loss)
    """
    def multi_loss(self,  ys_true, ys_pred):
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        i=0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision =keras.backend.exp(-log_var[0]) 
            if i==0:
                pred_loss = bbce(y_true, y_pred)
                term = main_task_weight*precision*pred_loss + main_task_weight*0.5 * log_var[0]  
                #term=tf.Print(keras.backend.mean(term), [keras.backend.mean(term)], 'mean bbce: ')
            #elif i==1:
                # I need to find a better way for this
            #    pred_loss = keras.losses.categorical_crossentropy(y_true, y_pred)
                #keras_mse(y_true, y_pred)
            #    term =  domain_weight * precision * pred_loss + domain_weight * log_var[0]
                #term=tf.Print(keras.backend.mean(term), [keras.backend.mean(term)], 'mean cce: ')
            else:
                pred_loss = keras_mse(y_true, y_pred)
                #pred_loss=tf.Print(pred_loss, [pred_loss], 'MSE: ')
                term = 0.5 * precision * pred_loss + 0.5 * log_var[0]
            loss+=term
            term = 0.
            i+=1
        return keras.backend.mean(loss)
    
    def call(self, inputs):
        ys_true = inputs[:self.nb_outputs]
        ys_pred = inputs[self.nb_outputs:]
        loss = self.multi_loss(ys_true, ys_pred)
        self.add_loss(loss, inputs=inputs)
        return keras.backend.concatenate(inputs, -1)

"""
EVALUATION FUNCTIONs
"""
def accuracy_domain(y_true,y_pred):
    y_p_r=np.round(y_pred)
    acc = np.equal(y_p_r, y_true)**1.
    acc = np.mean(np.float32(acc))
    return acc
def my_sigmoid(x):
    return 1 / (1 + np.exp(-x))
def my_accuracy_np(y_true, y_pred):
    sliced_y_pred = my_sigmoid(y_pred)
    y_pred_rounded = np.round(sliced_y_pred)
    acc = np.equal(y_pred_rounded, y_true)**1.
    acc = np.mean(np.float32(acc))
    return acc
def r_square_np(y_true, y_pred):
    SS_res =  np.sum(np.square(y_true - y_pred))
    SS_tot = np.sum(np.square(y_true - np.mean(y_true)))
    r2_mine=( 1 - SS_res/(SS_tot + keras.backend.epsilon()) )
    return ( 1 - SS_res/(SS_tot + keras.backend.epsilon()) )

global report_val_acc 
global report_val_r2
global report_val_mse
report_val_acc=[]
report_val_r2=[]
report_val_mse=[]

In [6]:
base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
layers_list=['conv2d_92', 'conv2d_93', 'conv2d_88', 'conv2d_89', 'conv2d_86']
#layers_list=[]
for i in range(len(base_model.layers[:])):
    layer=base_model.layers[i]
    if layer.name in layers_list:
        print layer.name
        layer.trainable=True
    else:
        layer.trainable = False
feature_output=base_model.layers[-1].output
gap_layer_output = keras.layers.GlobalAveragePooling2D()(feature_output)
feature_output = keras.layers.Dense(2048, activation='relu', name='finetuned_features1',kernel_regularizer=keras.regularizers.l2(0.01))(gap_layer_output) 
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
feature_output = keras.layers.Dense(512, activation='relu', name='finetuned_features2',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
feature_output = keras.layers.Dense(256, activation='relu', name='finetuned_features3',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
finetuning = keras.layers.Dense(1,name='predictions')(feature_output)
regression_output = keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='concept_regressor')(gap_layer_output)
model = keras.Model(input=base_model.input, output=[finetuning, regression_output])

conv2d_88
conv2d_89
conv2d_92
conv2d_93
conv2d_86


In [7]:
main_task_weight=1. 
domain_weight = 1. #e-100
t_m = get_trainable_model(model)

In [8]:
t_m.load_weights('{}/best_model.h5'.format(model_folder))

In [9]:
baseline=get_baseline_model()
baseline.load_weights('../../results/BASEL/best_model.h5')

In [10]:
len(data_list)

151697

In [1]:
test_list

NameError: name 'test_list' is not defined

In [11]:
type_='1139'
for layer in ['mixed10','finetuned_features1', 'finetuned_features2', 'finetuned_features3']:
    print layer
    get_guided_layer_output = K.function([model.layers[0].input],
                                  [model.get_layer(layer).output])
    get_base_layer_output = K.function([baseline.layers[0].input],
                                  [baseline.get_layer(layer).output])
    t_gen=DataGenerator(data_list, concept=CONCEPT, batch_size=BATCH_SIZE, data_type=1)
    N_SAMPLES=len(data_list)/2
    shape=int(model.get_layer(layer).output.shape[1])
    #import pdb; pdb.set_trace()
    if layer=='mixed10':
        shape=51200
    feats = np.zeros((N_SAMPLES, shape))
    i=0
    labs=np.zeros(N_SAMPLES)
    cvals=np.zeros(N_SAMPLES)
    #idxs = np.random.choice(np.arange(len(data_list)), N_SAMPLES, repetition=False)
    batch_size=32
    while i<=N_SAMPLES:
        #import pdb; pdb.set_trace()
        print i
        input_,_=t_gen.__getitem__(i//batch_size)
        try:
            feats[i:i+batch_size, :] = get_guided_layer_output([input_[0]])[0].reshape(batch_size,-1)
            labs[i:i+batch_size]=input_[1]
            cvals[i:i+batch_size]=input_[2]
            i+=batch_size
            
        except:
            #import pdb; pdb.set_trace()
            break
    np.save('{}/{}features{}.npy'.format(model_folder,type_, layer), feats)
    np.save('{}/{}labels{}.npy'.format(model_folder, type_,layer), labs)
    np.save('{}/{}cvalues{}.npy'.format(model_folder,type_, layer), cvals)
    t_gen=DataGenerator(data_list, concept=CONCEPT, batch_size=BATCH_SIZE, data_type=1)
    #N_SAMPLES=5000
    shape=int(model.get_layer(layer).output.shape[1])
    if layer=='mixed10':
        shape=51200
    feats = np.zeros((N_SAMPLES, shape))
    i=0
    labs=np.zeros(N_SAMPLES)
    cvals=np.zeros(N_SAMPLES)
    batch_size=32
    while i+batch_size<=N_SAMPLES:
        #import pdb; pdb.set_trace()
        print i
        input_,_=t_gen.__getitem__(i//batch_size)
        try:
            feats[i:i+batch_size, :] = get_base_layer_output([input_[0]])[0].reshape(batch_size,-1)
            labs[i:i+batch_size]=input_[1]
            cvals[i:i+batch_size]=input_[2]
            i+=batch_size
        except:
            i+=batch_size
            break
    np.save('{}/{}base_features{}.npy'.format(model_folder,type_,layer), feats)
    np.save('{}/{}base_labels{}.npy'.format(model_folder,type_,layer), labs)
    np.save('{}/{}base_cvalues{}.npy'.format(model_folder,type_,layer), cvals)


mixed10
0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
4416
4448
4480
4512
4544
4576
4608
4640
4672
4704
4736
4768
4800
4832
4864
4896
4928
4960
4992
5024
5056
5088
5120
5152
5184
5216
5248
5280
5312
5344
5376
5408
5440
5472
5504
5536
5568
5600
5632
5664
5696
5728
5760
5792
5824
5856
5888
5920
5952
5984
6016
6048
6080
6112
6144
6176
6208
6240
6272
6304
6336
6368
6400
6432
6464
6496
6528
6560

45536
45568
45600
45632
45664
45696
45728
45760
45792
45824
45856
45888
45920
45952
45984
46016
46048
46080
46112
46144
46176
46208
46240
46272
46304
46336
46368
46400
46432
46464
46496
46528
46560
46592
46624
46656
46688
46720
46752
46784
46816
46848
46880
46912
46944
46976
47008
47040
47072
47104
47136
47168
47200
47232
47264
47296
47328
47360
47392
47424
47456
47488
47520
47552
47584
47616
47648
47680
47712
47744
47776
47808
47840
47872
47904
47936
47968
48000
48032
48064
48096
48128
48160
48192
48224
48256
48288
48320
48352
48384
48416
48448
48480
48512
48544
48576
48608
48640
48672
48704
48736
48768
48800
48832
48864
48896
48928
48960
48992
49024
49056
49088
49120
49152
49184
49216
49248
49280
49312
49344
49376
49408
49440
49472
49504
49536
49568
49600
49632
49664
49696
49728
49760
49792
49824
49856
49888
49920
49952
49984
50016
50048
50080
50112
50144
50176
50208
50240
50272
50304
50336
50368
50400
50432
50464
50496
50528
50560
50592
50624
50656
50688
50720
50752
50784
50816
5084

15232
15264
15296
15328
15360
15392
15424
15456
15488
15520
15552
15584
15616
15648
15680
15712
15744
15776
15808
15840
15872
15904
15936
15968
16000
16032
16064
16096
16128
16160
16192
16224
16256
16288
16320
16352
16384
16416
16448
16480
16512
16544
16576
16608
16640
16672
16704
16736
16768
16800
16832
16864
16896
16928
16960
16992
17024
17056
17088
17120
17152
17184
17216
17248
17280
17312
17344
17376
17408
17440
17472
17504
17536
17568
17600
17632
17664
17696
17728
17760
17792
17824
17856
17888
17920
17952
17984
18016
18048
18080
18112
18144
18176
18208
18240
18272
18304
18336
18368
18400
18432
18464
18496
18528
18560
18592
18624
18656
18688
18720
18752
18784
18816
18848
18880
18912
18944
18976
19008
19040
19072
19104
19136
19168
19200
19232
19264
19296
19328
19360
19392
19424
19456
19488
19520
19552
19584
19616
19648
19680
19712
19744
19776
19808
19840
19872
19904
19936
19968
20000
20032
20064
20096
20128
20160
20192
20224
20256
20288
20320
20352
20384
20416
20448
20480
20512
2054

58944
58976
59008
59040
59072
59104
59136
59168
59200
59232
59264
59296
59328
59360
59392
59424
59456
59488
59520
59552
59584
59616
59648
59680
59712
59744
59776
59808
59840
59872
59904
59936
59968
60000
60032
60064
60096
60128
60160
60192
60224
60256
60288
60320
60352
60384
60416
60448
60480
60512
60544
60576
60608
60640
60672
60704
60736
60768
60800
60832
60864
60896
60928
60960
60992
61024
61056
61088
61120
61152
61184
61216
61248
61280
61312
61344
61376
61408
61440
61472
61504
61536
61568
61600
61632
61664
61696
61728
61760
61792
61824
61856
61888
61920
61952
61984
62016
62048
62080
62112
62144
62176
62208
62240
62272
62304
62336
62368
62400
62432
62464
62496
62528
62560
62592
62624
62656
62688
62720
62752
62784
62816
62848
62880
62912
62944
62976
63008
63040
63072
63104
63136
63168
63200
63232
63264
63296
63328
63360
63392
63424
63456
63488
63520
63552
63584
63616
63648
63680
63712
63744
63776
63808
63840
63872
63904
63936
63968
64000
64032
64064
64096
64128
64160
64192
64224
6425

28576
28608
28640
28672
28704
28736
28768
28800
28832
28864
28896
28928
28960
28992
29024
29056
29088
29120
29152
29184
29216
29248
29280
29312
29344
29376
29408
29440
29472
29504
29536
29568
29600
29632
29664
29696
29728
29760
29792
29824
29856
29888
29920
29952
29984
30016
30048
30080
30112
30144
30176
30208
30240
30272
30304
30336
30368
30400
30432
30464
30496
30528
30560
30592
30624
30656
30688
30720
30752
30784
30816
30848
30880
30912
30944
30976
31008
31040
31072
31104
31136
31168
31200
31232
31264
31296
31328
31360
31392
31424
31456
31488
31520
31552
31584
31616
31648
31680
31712
31744
31776
31808
31840
31872
31904
31936
31968
32000
32032
32064
32096
32128
32160
32192
32224
32256
32288
32320
32352
32384
32416
32448
32480
32512
32544
32576
32608
32640
32672
32704
32736
32768
32800
32832
32864
32896
32928
32960
32992
33024
33056
33088
33120
33152
33184
33216
33248
33280
33312
33344
33376
33408
33440
33472
33504
33536
33568
33600
33632
33664
33696
33728
33760
33792
33824
33856
3388

...

25024
25056
25088
25120
25152
25184
25216
25248
25280
25312
25344
25376
25408
25440
25472
25504
25536
25568
25600
25632
25664
25696
25728
25760
25792
25824
25856
25888
25920
25952
25984
26016
26048
26080
26112
26144
26176
26208
26240
26272
26304
26336
26368
26400
26432
26464
26496
26528
26560
26592
26624
26656
26688
26720
26752
26784
26816
26848
26880
26912
26944
26976
27008
27040
27072
27104
27136
27168
27200
27232
27264
27296
27328
27360
27392
27424
27456
27488
27520
27552
27584
27616
27648
27680
27712
27744
27776
27808
27840
27872
27904
27936
27968
28000
28032
28064
28096
28128
28160
28192
28224
28256
28288
28320
28352
28384
28416
28448
28480
28512
28544
28576
28608
28640
28672
28704
28736
28768
28800
28832
28864
28896
28928
28960
28992
29024
29056
29088
29120
29152
29184
29216
29248
29280
29312
29344
29376
29408
29440
29472
29504
29536
29568
29600
29632
29664
29696
29728
29760
29792
29824
29856
29888
29920
29952
29984
30016
30048
30080
30112
30144
30176
30208
30240
30272
30304
3033

68736
68768
68800
68832
68864
68896
68928
68960
68992
69024
69056
69088
69120
69152
69184
69216
69248
69280
69312
69344
69376
69408
69440
69472
69504
69536
69568
69600
69632
69664
69696
69728
69760
69792
69824
69856
69888
69920
69952
69984
70016
70048
70080
70112
70144
70176
70208
70240
70272
70304
70336
70368
70400
70432
70464
70496
70528
70560
70592
70624
70656
70688
70720
70752
70784
70816
70848
70880
70912
70944
70976
71008
71040
71072
71104
71136
71168
71200
71232
71264
71296
71328
71360
71392
71424
71456
71488
71520
71552
71584
71616
71648
71680
71712
71744
71776
71808
71840
71872
71904
71936
71968
72000
72032
72064
72096
72128
72160
72192
72224
72256
72288
72320
72352
72384
72416
72448
72480
72512
72544
72576
72608
72640
72672
72704
72736
72768
72800
72832
72864
72896
72928
72960
72992
73024
73056
73088
73120
73152
73184
73216
73248
73280
73312
73344
73376
73408
73440
73472
73504
73536
73568
73600
73632
73664
73696
73728
73760
73792
73824
73856
73888
73920
73952
73984
74016
7404

38368
38400
38432
38464
38496
38528
38560
38592
38624
38656
38688
38720
38752
38784
38816
38848
38880
38912
38944
38976
39008
39040
39072
39104
39136
39168
39200
39232
39264
39296
39328
39360
39392
39424
39456
39488
39520
39552
39584
39616
39648
39680
39712
39744
39776
39808
39840
39872
39904
39936
39968
40000
40032
40064
40096
40128
40160
40192
40224
40256
40288
40320
40352
40384
40416
40448
40480
40512
40544
40576
40608
40640
40672
40704
40736
40768
40800
40832
40864
40896
40928
40960
40992
41024
41056
41088
41120
41152
41184
41216
41248
41280
41312
41344
41376
41408
41440
41472
41504
41536
41568
41600
41632
41664
41696
41728
41760
41792
41824
41856
41888
41920
41952
41984
42016
42048
42080
42112
42144
42176
42208
42240
42272
42304
42336
42368
42400
42432
42464
42496
42528
42560
42592
42624
42656
42688
42720
42752
42784
42816
42848
42880
42912
42944
42976
43008
43040
43072
43104
43136
43168
43200
43232
43264
43296
43328
43360
43392
43424
43456
43488
43520
43552
43584
43616
43648
4368

7680
7712
7744
7776
7808
7840
7872
7904
7936
7968
8000
8032
8064
8096
8128
8160
8192
8224
8256
8288
8320
8352
8384
8416
8448
8480
8512
8544
8576
8608
8640
8672
8704
8736
8768
8800
8832
8864
8896
8928
8960
8992
9024
9056
9088
9120
9152
9184
9216
9248
9280
9312
9344
9376
9408
9440
9472
9504
9536
9568
9600
9632
9664
9696
9728
9760
9792
9824
9856
9888
9920
9952
9984
10016
10048
10080
10112
10144
10176
10208
10240
10272
10304
10336
10368
10400
10432
10464
10496
10528
10560
10592
10624
10656
10688
10720
10752
10784
10816
10848
10880
10912
10944
10976
11008
11040
11072
11104
11136
11168
11200
11232
11264
11296
11328
11360
11392
11424
11456
11488
11520
11552
11584
11616
11648
11680
11712
11744
11776
11808
11840
11872
11904
11936
11968
12000
12032
12064
12096
12128
12160
12192
12224
12256
12288
12320
12352
12384
12416
12448
12480
12512
12544
12576
12608
12640
12672
12704
12736
12768
12800
12832
12864
12896
12928
12960
12992
13024
13056
13088
13120
13152
13184
13216
13248
13280
13312
13344
13376

51776
51808
51840
51872
51904
51936
51968
52000
52032
52064
52096
52128
52160
52192
52224
52256
52288
52320
52352
52384
52416
52448
52480
52512
52544
52576
52608
52640
52672
52704
52736
52768
52800
52832
52864
52896
52928
52960
52992
53024
53056
53088
53120
53152
53184
53216
53248
53280
53312
53344
53376
53408
53440
53472
53504
53536
53568
53600
53632
53664
53696
53728
53760
53792
53824
53856
53888
53920
53952
53984
54016
54048
54080
54112
54144
54176
54208
54240
54272
54304
54336
54368
54400
54432
54464
54496
54528
54560
54592
54624
54656
54688
54720
54752
54784
54816
54848
54880
54912
54944
54976
55008
55040
55072
55104
55136
55168
55200
55232
55264
55296
55328
55360
55392
55424
55456
55488
55520
55552
55584
55616
55648
55680
55712
55744
55776
55808
55840
55872
55904
55936
55968
56000
56032
56064
56096
56128
56160
56192
56224
56256
56288
56320
56352
56384
56416
56448
56480
56512
56544
56576
56608
56640
56672
56704
56736
56768
56800
56832
56864
56896
56928
56960
56992
57024
57056
5708